In [1]:
import numpy as np
import pandas as pd
import requests
import json

import os
import ta


pd.set_option('display.max_columns', 100)


: 

: 

In [5]:
EXCHANGE_ID = "2eccdb22-ec97-4799-9755-77c976991af8"

# BTCUSDT_DAY_ID = "2b043a89-de72-4920-a375-fb5991ac405e"
BTCUSDT_HOUR_ID = "2b043a89-de72-4920-a375-fb5991ac405e"
BTCUSDT_MINUTE_ID = ""

ETHUSDT_DAY_ID = "9d14a067-c116-4cb9-b33e-c8a5344a2337"
ETHUSDT_HOUR_ID = ""
ETHUSDT_MINUTE_ID = ""

BNBUSDT_DAY_ID = "d2ef1f0c-8842-4c75-8dc1-61e5410e511b"
BNBUSDT_HOUR_ID = ""
BNBUSDT_MINUTE_ID = ""




ASSET_ID = BTCUSDT_HOUR_ID

base_url = "http://172.24.100.128:5000/"
# base_url = "http://localhost:5000/"


PROXIES = {
    
   'http': 'http://discproxy.virtual.uniandes.edu.co:443',
}


In [6]:
# Method to make API request to retrieve prices for a given asset ID between two specified Unix timestamps
def api_request_get_prices_between_unix_time(base_url, asset_id, unix_time_start, unix_time_end):

    # try:
    # Make GET request to API endpoint with query parameters for start and end Unix timestamps
    response = requests.get(base_url + "assets/" + asset_id + "/indicators_unix_between/",
                            params={'unix_time_start': unix_time_start, 'unix_time_end': unix_time_end},proxies=PROXIES)

    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        # print("get prices between status code: "+str(response.status_code))
        # Convert JSON response to Python dictionary
        json_data = json.loads(response.content)
        # Normalize the dictionary and convert it to a Pandas DataFrame
        df = pd.json_normalize(json_data)

        # drop columns that are not needed
        df.drop(columns=['updated_at', 'created_at', 'date_time_gmt_5','ignore','asset_id'], inplace=True)

        # Return the DataFrame containing the retrieved price data
        return df

    else:
        # print("Failed api_request_get_prices_between_unix_time")
        print("get prices between status code: "+str(response.status_code))
        json_data = json.loads(response.content)
        print(json_data)
        return json_data
    


def api_request_get_asset_from_asset_id(base_url, exchange_id, asset_id):
    # try:
    # Make GET request to API endpoint

    response = requests.get(base_url + "exchanges/" +
                            exchange_id + "/asset/"+asset_id,proxies=PROXIES)

    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        print("get asset status code: "+str(response.status_code))
        # Convert JSON response to Python dictionary
        json_data = json.loads(response.content)
        # Normalize the dictionary and convert it to a Pandas DataFrame
        df = pd.json_normalize(json_data)
        # Return the Unix timestamp of the last recorded price for the specified asset ID

        print("symbol: " + df["symbol"].iloc[0])
        print("interval: " + df["interval"].iloc[0])
        return df
    else:
        print(response.content)
        json_data = json.loads(response.content)
        df = pd.json_normalize(json_data)
        return df



# Method to make API request to retrieve prices for a given asset ID between two specified Unix timestamps
def api_request_get_all_indicators_from_price(base_url, price_id,):

    # try:
    # Make GET request to API endpoint with query parameters for start and end Unix timestamps
    response = requests.get(base_url + "prices/" + price_id + "/indicators/",proxies=PROXIES)

    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        # Convert JSON response to Python dictionary
        json_data = json.loads(response.content)
        # Normalize the dictionary and convert it to a Pandas DataFrame
        df = pd.json_normalize(json_data)

        if df.empty:
            return df
        else:
        
            df.drop(columns=['id', 'created_at', 'updated_at','date_time_utc','unix_time','date_time_gmt_5'], inplace=True)

            df.set_index('price_id', inplace=True)

            df = df.replace(0, np.nan)

            return df

    else:
        print("get indicators status code: "+str(response.status_code))
        json_data = json.loads(response.content)
        print(json_data)
        return json_data
    

def get_indicators_and_merge(base_url,exchange_id, asset_id, unix_time_start, unix_time_end):


    # Get the asset from the API
    asset = api_request_get_asset_from_asset_id(base_url,exchange_id,asset_id)

    interval = asset["interval"].iloc[0]
    
    if interval == "minute":
        first_indicator_time = unix_time_start - 6000

    if interval == "hour":
        first_indicator_time = unix_time_start - 360000

    if interval == "day":
        first_indicator_time = unix_time_start - 8640000


    df = api_request_get_prices_between_unix_time(base_url, asset_id, first_indicator_time, unix_time_end)

    print(df.info())
    
    df_prices_wit_indicators_all = add_indicators(df)

    df_prices_wit_indicators = df_prices_wit_indicators_all.loc[df_prices_wit_indicators_all['unix_time'] >= unix_time_start]

    df_assets = pd.DataFrame()

    for i in df['id']:

        df2 = api_request_get_all_indicators_from_price(base_url, i)

        if not df2.empty:
            df_assets = pd.concat([df_assets, df2])
        
    if not df_assets.empty:
        df_merge = pd.merge(df_prices_wit_indicators, df_assets, left_on='id', right_on='price_id', how='left')
    else:
        df_merge = df_prices_wit_indicators

    return df_merge


# method to add the thecnic indicators to the dataframe
def add_indicators(df):

    # momentum
    df['ao'] = ta.momentum.awesome_oscillator(
        high=df['high_price'], low=df['low_price'])
    df['kama'] = ta.momentum.kama(close=df['close_price'])
    df['ppo'] = ta.momentum.ppo(close=df['close_price'])
    df['pvo'] = ta.momentum.pvo(volume=df['volume'])
    df['roc'] = ta.momentum.roc(close=df['close_price'])
    df['rsi'] = ta.momentum.rsi(close=df['close_price'])
    df['stochrsi'] = ta.momentum.stochrsi(close=df['close_price'])
    df['stoch'] = ta.momentum.stoch(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['tsi'] = ta.momentum.tsi(close=df['close_price'])
    df['uo'] = ta.momentum.ultimate_oscillator(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['wri'] = ta.momentum.williams_r(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    # volume
    df['accdist'] = ta.volume.acc_dist_index(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])
    df['cmf'] = ta.volume.chaikin_money_flow(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])
    df['emv'] = ta.volume.ease_of_movement(
        high=df['high_price'], low=df['low_price'], volume=df['volume'])
    df['fi'] = ta.volume.force_index(
        close=df['close_price'], volume=df['volume'])
    df['mfi'] = ta.volume.money_flow_index(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])
    df['nvi'] = ta.volume.negative_volume_index(
        close=df['close_price'], volume=df['volume'])
    df['obv'] = ta.volume.on_balance_volume(
        close=df['close_price'], volume=df['volume'])
    df['smaemv'] = ta.volume.sma_ease_of_movement(
        high=df['high_price'], low=df['low_price'], volume=df['volume'])
    df['vpt'] = ta.volume.volume_price_trend(
        close=df['close_price'], volume=df['volume'])
    df['vwap'] = ta.volume.volume_weighted_average_price(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])

    # volatility
    df['atr'] = ta.volatility.average_true_range(
        high=df['high_price'], low=df['low_price'], close=df['close_price'])
    df['ulcer'] = ta.volatility.ulcer_index(close=df['close_price'])

    df['bbh'] = ta.volatility.bollinger_hband(close=df['close_price'])
    df['bbl'] = ta.volatility.bollinger_lband(close=df['close_price'])
    df['bbhi'] = ta.volatility.bollinger_hband_indicator(
        close=df['close_price'])
    df['bbli'] = ta.volatility.bollinger_lband_indicator(
        close=df['close_price'])
    df['bbmavg'] = ta.volatility.bollinger_mavg(close=df['close_price'])
    df['bb_pb'] = ta.volatility.bollinger_pband(close=df['close_price'])
    df['bb_wb'] = ta.volatility.bollinger_wband(close=df['close_price'])

    df['dchb'] = ta.volatility.donchian_channel_hband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dclb'] = ta.volatility.donchian_channel_lband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dcmb'] = ta.volatility.donchian_channel_mband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dcpb'] = ta.volatility.donchian_channel_pband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dcwb'] = ta.volatility.donchian_channel_wband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    df['kchb'] = ta.volatility.keltner_channel_hband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kclb'] = ta.volatility.keltner_channel_lband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kchbi'] = ta.volatility.keltner_channel_hband_indicator(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kclbi'] = ta.volatility.keltner_channel_lband_indicator(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kcmb'] = ta.volatility.keltner_channel_mband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kcpb'] = ta.volatility.keltner_channel_pband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kcwb'] = ta.volatility.keltner_channel_wband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    # trend

    # En dos de estos tres hay algo que esta botando warning
    df['adx'] = ta.trend.adx(close=df['close_price'],
                             high=df['high_price'], low=df['low_price'])
    df['adx_neg'] = ta.trend.adx_neg(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['adx_pos'] = ta.trend.adx_pos(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    df['aroon_up'] = ta.trend.aroon_up(close=df['close_price'])
    df['aroon_down'] = ta.trend.aroon_down(close=df['close_price'])

    df['cci'] = ta.trend.cci(close=df['close_price'],
                             high=df['high_price'], low=df['low_price'])
    df['dpo'] = ta.trend.dpo(close=df['close_price'])
    df['kst'] = ta.trend.kst(close=df['close_price'])
    df['kst_sig'] = ta.trend.kst_sig(close=df['close_price'])
    df['ema'] = ta.trend.ema_indicator(close=df['close_price'])

    df['ichimoku_a'] = ta.trend.ichimoku_a(
        high=df['high_price'], low=df['low_price'])
    df['ichimoku_b'] = ta.trend.ichimoku_b(
        high=df['high_price'], low=df['low_price'])
    df['ichimoku_base_line'] = ta.trend.ichimoku_base_line(
        high=df['high_price'], low=df['low_price'])
    df['ichimoku_conversion_line'] = ta.trend.ichimoku_conversion_line(
        high=df['high_price'], low=df['low_price'])

    df['macd'] = ta.trend.macd(close=df['close_price'])
    df['macd_diff'] = ta.trend.macd_diff(close=df['close_price'])
    df['macd_signal'] = ta.trend.macd_signal(close=df['close_price'])

    df['mi'] = ta.trend.mass_index(high=df['high_price'], low=df['low_price'])
    df['sma'] = ta.trend.sma_indicator(close=df['close_price'])
    df['wma'] = ta.trend.wma_indicator(close=df['close_price'])
    df['stc'] = ta.trend.stc(close=df['close_price'])
    df['trix'] = ta.trend.trix(close=df['close_price'])

    df['vi_pos'] = ta.trend.vortex_indicator_pos(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['vi_neg'] = ta.trend.vortex_indicator_neg(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    return df

In [7]:
df_prueba = get_indicators_and_merge(base_url, EXCHANGE_ID, ASSET_ID, 1681603200, 1681948800)
df_prueba.sort_values(by='unix_time', ascending=True, inplace=True)
df_prueba.tail(15)

b'{"error message":"The exchange with the given id was not found"}'


KeyError: 'interval'

In [6]:
df_prueba.tail(15)


,low_price,id,high_price,volume,unix_time,qav,date_time_utc,num_trades,taker_base_vol,open_price,taker_quote_vol,close_price,ao,kama,ppo,pvo,roc,rsi,stochrsi,stoch,tsi,uo,wri,accdist,cmf,emv,fi,mfi,nvi,obv,smaemv,vpt,vwap,atr,ulcer,bbh,bbl,bbhi,bbli,bbmavg,bb_pb,bb_wb,dchb,dclb,dcmb,dcpb,dcwb,kchb,kclb,kchbi,kclbi,kcmb,kcpb,kcwb,adx,adx_neg,adx_pos,aroon_up,aroon_down,cci,dpo,kst,kst_sig,ema,ichimoku_a,ichimoku_b,ichimoku_base_line,ichimoku_conversion_line,macd,macd_diff,macd_signal,mi,sma,wma,stc,trix,vi_pos,vi_neg,natural_gas_price,soybean_price,us_2_year_treasury_price,cotton_price,oats_price,usbond_price,gold_price,coffee_price,spy500_price,sugar_price,dow_jones_price,cocoa_price,nasdaq_price,rice_price,russell_2000_price,corn_price,us_10_year_treasury_price,silver_price,wheat_price,us_5_year_treasury_price
2,2072.72,edbd17ed-a887-4c36-93e3-8e9e5e2e6ec5,2141.54,319880.4393,1681603200,6.730346e+08,2023-04-16T00:00:00+00:00,529231,162518.1328,2090.61,3.419549e+08,2118.67,212.288176,1844.994908,4.126543,-5.022653,35.414616,59.060723,0.883748,97.041971,9.006740,57.055110,-2.958029,4.889607e+06,0.138113,6.645226e+06,2.665260e+07,66.792578,905.969741,1.213047e+07,204442.411667,33168.594133,1783.525115,169.142110,17.026544,2162.540432,1460.981568,0.0,0.0,1811.7610,0.937467,38.722484,2141.54,1368.39,1931.770,0.945488,22.162131,1879.369000,1737.134000,1.0,0.0,1808.251500,2.682434,7.865886,11.866197,29.229721,46.597839,100.0,56.0,156.046334,-385.3210,100.033336,49.336893,1883.389240,1853.5600,1754.965,1754.965,1952.155,74.638854,27.558707,47.080146,24.505381,1833.907500,1946.210444,97.387026,0.370395,0.888805,0.811696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2056.25,4b3a5949-55c4-47aa-bbf2-b5f5ea0e9cf7,2120.51,426972.7071,1681689600,8.894333e+08,2023-04-17T00:00:00+00:00,674817,206218.4799,2118.66,4.296169e+08,2074.00,245.304618,1858.694059,4.611438,-6.234486,45.396932,57.573438,0.807479,91.264308,10.045103,58.398120,-8.735692,4.698512e+06,0.100025,-2.821902e+05,2.012039e+07,65.638714,905.969741,1.170350e+07,614396.476200,48780.866661,1807.400982,161.650530,15.815871,2194.248494,1469.300506,0.0,0.0,1831.7745,0.834128,39.576268,2141.54,1368.39,1952.155,0.821686,19.654198,1900.510500,1757.773500,1.0,0.0,1829.142000,2.215449,7.803495,12.568802,29.127523,45.274812,96.0,52.0,124.264041,-181.8145,108.802907,58.254327,1912.713972,1853.5600,1754.965,1754.965,1952.155,84.315467,29.788256,54.527211,24.483520,1887.870833,1978.839111,98.693513,0.430996,0.906287,0.772811,2.275,1517.00,102.972656,83.300003,342.00,129.75000,1994.199951,201.000000,4176.75,24.440001,34129.0,3003.0,13186.50,1733.0,1814.199951,676.50,114.359375,25.052999,889.75,109.101562
0,2051.00,fb7d765a-c65a-4854-b91d-59f946e4589c,2125.00,414244.6135,1681776000,8.672915e+08,2023-04-18T00:00:00+00:00,636801,205943.0188,2073.99,4.311824e+08,2103.50,141.263618,1764.508267,3.272705,-2.497131,22.637329,63.837997,1.000000,96.999721,8.221780,67.535378,-3.000279,4.464197e+06,0.119647,3.486575e+06,2.778547e+07,44.224982,1082.003694,9.819734e+06,334642.582606,73677.090940,1708.715477,139.875645,11.653896,2001.347719,1410.957281,1.0,0.0,1706.1525,1.173025,34.603615,2125.00,1408.00,1766.700,0.969997,41.133531,1778.875833,1630.862833,1.0,0.0,1704.869333,3.193214,8.681780,11.678788,25.179902,45.655227,100.0,36.0,234.093857,115.3675,46.138462,8.094086,1793.354901,1766.6000,1693.150,1766.500,1766.700,56.831286,39.285357,17.545930,24.355977,1745.508333,1830.695778,93.709317,0.117537,0.876959,0.785816,2.366,1519.25,102.949219,84.550003,347.50,130.03125,2007.400024,205.250000,4180.00,24.540001,34124.0,3054.0,13193.00,1734.0,1806.000000,677.50,114.484375,25.247999,881.25,109.164062
4,1923.03,567b7316-ae9a-4b56-9329-ffdf59cfcf0c,2104.60,775389.0602,1681862400,1.547934e+09,2023-04-19T00:00:00+00:00,1116849,381226.9641,2103.51,7.604785e+08,1933.73,162.352824,1861.509848,3.514845,-3.173367,6.805817,52.653510,0.203864,45.135570,7.633135,49.028783,-54.864430,4.41790

In [7]:
df_prueba.to_csv('prueba.csv', index=False)